### 1.pymongo 

In [2]:
import sys
import pandas as pd
import json
from pymongo import MongoClient

In [3]:
database='stocks'
collection='price'
class MongoBase:
    def __init__(self,database,collection):
        self.data=database
        self.coll=collection
        self.OpenDB()
    def OpenDB(self):
        '''
        open or create database and collection
        '''
        user='******'
        passwd='******'
        host='******'
        port='******'
        auth_db='******'
        #uri = "mongodb://"+user+":"+passwd+"@"+host+":"+port+"/"+auth_db+"?authMechanism=SCRAM-SHA-1"
        self.Client = MongoClient('localhost',27017)#接口
        self.db = self.Client[self.data]#数据库
        self.collection=self.db[self.coll]#colle
    def closeDB(self):
        self.Client.close()

In [7]:
col=MongoBase(database,collection).collection

In [12]:
for i in col.find().limit(2):
    print(i)

{'_id': ObjectId('5ab9ebe1b1ddee43444b9421'), 'open': 3.36, 'high': 3.41, 'close': 3.41, 'low': 3.31, 'volume': 36288.92, 'price_change': 0.01, 'p_change': 0.29, 'ma5': 3.53, 'ma10': 3.579, 'ma20': 3.602, 'v_ma5': 45378.39, 'v_ma10': 36493.33, 'v_ma20': 30986.53, 'turnover': 0.39, 'date': '2018-03-26', 'sid': '601616'}
{'_id': ObjectId('5ab9ebe1b1ddee43444b9422'), 'open': 3.56, 'high': 3.57, 'close': 3.4, 'low': 3.4, 'volume': 65957.11, 'price_change': -0.24, 'p_change': -6.59, 'ma5': 3.574, 'ma10': 3.609, 'ma20': 3.609, 'v_ma5': 42502.8, 'v_ma10': 36477.81, 'v_ma20': 30922.89, 'turnover': 0.7, 'date': '2018-03-23', 'sid': '601616'}


#### 1.  涨幅大于5%，代码等于000768，且日期大于2017-02-03且小于2018-01-01

In [20]:
query={'p_change':{'$gt':5},'date':{'$gt':'2017-02-03','$lt':'2018-01-01'},'sid':'000768'}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection):
    print(i)

{'date': '2017-08-04', 'sid': '000768'}
{'date': '2017-04-11', 'sid': '000768'}


#### 2.  涨幅大于5%，代码等于000768，或者日期大于2017-02-03

In [37]:
query={'p_change':{'$gt':5},'$or':[{'sid':'000768'},{'date':{'$lt':'2018-02-03'}}]}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2018-03-26', 'sid': '000768'}
{'date': '2018-02-27', 'sid': '000768'}
{'date': '2018-02-02', 'sid': '601899'}
{'date': '2018-01-25', 'sid': '601899'}
{'date': '2017-12-28', 'sid': '601899'}


#### 3. in not in

In [47]:
query={'p_change':{'$gt':5},'date':{'$in':['2018-03-26']}}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2018-03-26', 'sid': '000768'}


In [48]:
query={'p_change':{'$gt':5},'date':{'$nin':['2018-03-26']}}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2018-03-08', 'sid': '601899'}
{'date': '2018-02-27', 'sid': '000768'}
{'date': '2018-02-02', 'sid': '601899'}
{'date': '2018-01-25', 'sid': '601899'}
{'date': '2017-12-28', 'sid': '601899'}


In [60]:
query={'p_change':{'$all':[3]}}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2016-03-17', 'sid': '000768'}


In [62]:
query={'p_change':{'$ne':None}}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2018-03-26', 'sid': '601616'}
{'date': '2018-03-26', 'sid': '000768'}
{'date': '2018-03-26', 'sid': '300040'}
{'date': '2018-03-26', 'sid': '601899'}
{'date': '2018-03-23', 'sid': '601616'}


#### 4. 正则表达式

In [65]:
query={'sid':{'$regex':'000[\d]{3}'}}
projection = {'_id':False, 'sid':1,'date':1}
for i in col.find(query,projection).sort('date',-1).limit(5):
    print(i)

{'date': '2018-03-26', 'sid': '000768'}
{'date': '2018-03-23', 'sid': '000768'}
{'date': '2018-03-22', 'sid': '000768'}
{'date': '2018-03-21', 'sid': '000768'}
{'date': '2018-03-20', 'sid': '000768'}


#### 5. 目标是一个数组可以用where

In [78]:
MongoBase(database,collection).closeDB()